In [97]:
from qdrant_client import QdrantClient, models
from fastembed import TextEmbedding
import numpy as np

In [98]:
qd_client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

## Q1. Embedding the query

In [99]:
model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

In [100]:
q = 'I just discovered the course. Can I join now?'

In [101]:
embedding_q = next(model.embed([q]))

In [102]:
len(embedding_q)

512

In [103]:
embedding_q[0]

np.float64(-0.07639462602911798)

In [104]:
min_value = embedding_q.min()
min_value

np.float64(-0.11726373885183883)

**A1:** -0.11

## Q2. Cosine similarity with another vector

In [105]:
doc = "Can I still join the course after the start date?"

In [106]:
embedding_doc = next(model.embed([doc]))

In [107]:
cosine_sim = np.dot(embedding_q, embedding_doc)
print("Cosine similarity:", cosine_sim)

Cosine similarity: 0.9008528895674548


**A2:** 0.9

## Q3. Ranking by cosine


In [108]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [109]:
doc_texts = [doc['text'] for doc in documents]
doc_embeddings = list(model.embed(doc_texts))

In [110]:
V = np.stack(doc_embeddings)
cosines = V.dot(embedding_q)
best_idx = np.argmax(cosines)
best_idx, cosines

(np.int64(1),
 array([0.76296845, 0.81823782, 0.80853974, 0.71330788, 0.73044992]))

**A3:** 1

## Q4. Ranking by cosine, version two

In [111]:
full_texts = [doc['question'] + ' ' + doc['text'] for doc in documents]
full_embeddings = list(model.embed(full_texts))
V_full = np.stack(full_embeddings)
cosines_full = V_full.dot(embedding_q)
best_idx_full = np.argmax(cosines_full)
best_idx_full, cosines_full

(np.int64(0),
 array([0.85145432, 0.8436594 , 0.84082872, 0.77551577, 0.80860079]))

**A4:** 0

## Q5. Selecting the embedding model

In [112]:
models_list = TextEmbedding.list_supported_models()

In [113]:
dims = []
for model in models_list:
    dims.append(model['dim'])
    
models_list[np.array(dims).argmin()]

{'model': 'BAAI/bge-small-en',
 'sources': {'hf': 'Qdrant/bge-small-en',
  'url': 'https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz',
  '_deprecated_tar_struct': True},
 'model_file': 'model_optimized.onnx',
 'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
 'license': 'mit',
 'size_in_GB': 0.13,
 'additional_files': [],
 'dim': 384,
 'tasks': {}}

**A5:** 384

## Q6. Indexing with qdrant 

In [114]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [115]:
collection_name = "ml-zoomcamp-faq"
embedding_model = "BAAI/bge-small-en"
EMBEDDING_DIM = 384

In [116]:
qd_client.delete_collection(collection_name=collection_name)
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIM,
        distance=models.Distance.COSINE
    )
)

True

In [117]:
model = TextEmbedding(model_name=embedding_model)
points = []
for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = next(model.embed([text]))
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

qd_client.upsert(collection_name=collection_name, points=points)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [118]:
query = "I just discovered the course. Can I join now?"
query_vector = next(model.embed([query]))

In [119]:
results = qd_client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=1,
    with_payload=True
)


/tmp/ipykernel_992/940802027.py:1: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qd_client.search(


In [120]:
results[0].score

0.87031734

**A6:** 0.87